In [1]:
import pandas as pd
import numpy as np
import os
import glob
from time import perf_counter
import tqdm 

### data load

In [2]:
directory = os.getenv('HOME') + '/aiffel/aiffelthon'

In [3]:
os.listdir(directory +'/meta')

['reg_1.json',
 'meta_data_included_raw_v2.csv',
 '.ipynb_checkpoints',
 'meta_eng_raw_0306_v1.csv',
 'kor_text.xlsx',
 'reg_3.json',
 'meta_data_included_raw.csv',
 'meta_data_raw_eng_match_6m.csv',
 'kor_text.txt',
 'reg_4.json',
 'kw_t.json',
 'reg_5.json',
 'meta_data_raw_0306.csv',
 'reg_2.json']

In [4]:
df = pd.read_csv(directory + '/meta/meta_eng_raw_0306_v1.csv')

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (6,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 949888 entries, 0 to 949887
Data columns (total 25 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   text              949888 non-null  object
 1   dial              949888 non-null  object
 2   eng               949888 non-null  object
 3   title             949888 non-null  object
 4   creator           949888 non-null  object
 5   distributor       949706 non-null  object
 6   year              949888 non-null  object
 7   category          949706 non-null  object
 8   annotation_level  949888 non-null  object
 9   sampling          949417 non-null  object
 10  author            925576 non-null  object
 11  publisher         925576 non-null  object
 12  date              949888 non-null  object
 13  topic             949888 non-null  object
 14  age               949888 non-null  object
 15  sex               949888 non-null  object
 16  prin_res          949888 non-null  obj

In [6]:
df.reg.value_counts()

kw    345244
jj    203349
jd    174833
gs    114787
cc    111675
Name: reg, dtype: int64

## 가장 적합한 모델 선택

허깅페이스에서 테스트 해볼만한 6가지 모델을 추출하였다.

In [7]:
model_ckpt3 ='ddobokki/klue-roberta-small-nli-sts'
gpt_ckpt = 'skt/kogpt2-base-v2'

In [8]:
# ! pip install sentence_transformers

In [18]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer(model_ckpt3)
model1 = AutoModel.from_pretrained(model_ckpt3)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt3)

In [19]:
def get_sts(text,topic):
    # model loading
#     model = SentenceTransformer(model_ckpt)
    
    # get embedding
#     start_time = perf_counter()
    topic_embedding = model.encode(topic, convert_to_tensor=True)
    src_embedding = model.encode(text, convert_to_tensor=True)
    
    # 유사도 계산
    sts = util.pytorch_cos_sim(src_embedding, topic_embedding).to('cpu').numpy()
    
#     latency = perf_counter() - start_time
#     print(f'{latency * 1000:.3f} ms')
    return sts

In [20]:
stdns = df['stdn'].values
topics = df['topic'].values

In [21]:
input_val = tokenizer(stdns[0], return_tensors ='pt')

In [22]:
import torch
with torch.no_grad():
    output = model1(**input_val)
output[0].mean(axis=1).sum()

tensor(25.7711)

In [23]:
model.encode(stdns[0], show_progress_bar = True, batch_size =64, convert_to_tensor=True).sum()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor(25.7711, device='cuda:0')

In [30]:
scores = []
for i in range(14842 ,len(stdns)+14842,14842):
    cosine_score =get_sts(stdns[i-14842:i], topics[i-14842:i])
    for k in range(len(cosine_score)):
        scores.append(cosine_score[k][k])
    print(len(scores))

949888


In [31]:
print(len(scores))

949888


In [33]:
df['sts'] = pd.Series(scores)

In [39]:
topic_label = df.topic.unique()

In [46]:
enc = tokenizer.encode('기술발전')
tok = tokenizer.convert_ids_to_tokens(enc)
print(tok)

['[CLS]', '기술', '##발전', '[SEP]']


In [37]:
df.loc[:,['stdn','topic','sts']]

,stdn,topic,sts
0,생각이 쪼금씩 바뀌더라고,역사,0.056177
1,어 알겠는가 외국인들이 그래도 잘 적응하고,가족,-0.006247
2,아으 갑자기 예전에 맹장 뜯은 게 생각난다.,건강,0.174105
3,보면은 조금 품위 있게 나이 들어가야 되겠단 생각을 참 많이 해요.,스타일,0.120993
4,약간 조금 약간 맛이 쪼끔 거시기 하긴 한데 그래도,먹거리,0.256973
...,...,...,...
949883,뭐~ 맨날 집에서 인제 음식도 그냥 가까운 데 나가서 먹는 게 아니라 되게 시켜먹는...,다이어트,0.047894
949884,너는 만약에 그런 시대가 오면 네가 어떻게 활동할 것 같애?,4차산업혁명,0.142204
949885,하이브도 있고 #조석 작가 이제 조의 영역이라는 웹툰도 있고 마음의 소리도 있는데,만화,0.282759
949886,추석도 친척들도 사촌까지만 모이긴 했는데 이~ 다 모이니까 할 말도 많고 이~,반려동물,-0.043421


In [51]:
df.loc[df.text != df.stdn, ['text','stdn']]

,text,stdn
460,옛날에는 모이면 살고 흩어지면 죽는다 그랬는데 지금은 그 명언이 바까져 갖고/바뀌어서,옛날에는 모이면 살고 흩어지면 죽는다 그랬는데 지금은 그 명언이 바뀌어서
940,개수가 되려나 모르겄/모르겠다 감자 한 번 봐바야 겠네.,개수가 되려나모르겠다 감자 한 번 봐바야 겠네.
972,그랬는데 쫌 저희가 그냥 이렇게 금방금방 사기에는,그랬는데 조금 저희가 그냥 이렇게 금방금방 사기에는
1308,마르고 닳도록 읽어 가꼬/읽어가지고 책 겉이 다 뜯어졌네,마르고 닳도록 읽어가지고 책 겉이 다 뜯어졌네
1388,알겠지?,알았지?
...,...,...
947372,아이고 손이 왜 영 /이렇게 차가워요?,아이고 손이 왜 이렇게 차가워요?
948024,뻥 치시네 허멍 허니까/하니까 아니라고 진짜,뻥 치시네 하니까 아니라고 진짜
948862,그리고 계산기도 다 두드리더라구 그서 /그래서 우리 때랑 너무 많이 바뀐 거야,그리고 계산기도 다 두드리더라구 그래서 우리 때랑 너무 많이 바뀐 거야
948871,순직한 선배 형사 아덜인데 /아들인데 다섯 살 때부터 자기가 키우는 거야.,순직한 선배 형사 아들인데 다섯 살 때부터 자기가 키우는 거야.


In [67]:
# create a list of our conditions
conditions = [
    (df['sts'] <= 0),
    (df['sts'] > 0) & (df['sts'] <= 0.2),
    (df['sts'] > 0.2) & (df['sts'] <= 0.4),
    (df['sts'] > 0.4) & (df['sts'] <= 0.6),
    (df['sts'] > 0.6)
    ]

# create a list of the values we want to assign for each condition
values = ['very low(<=0)', 'low(0<=0.2)', 'mid(0.2<=0.4)', 'high(0.4<=0.6)','very high(0.6<)']

# create a new column and use np.select to assign values to it using our lists as arguments
df['sts_q'] = np.select(conditions, values)

In [68]:
df['sts_q'].value_counts()

low(0<=0.2)        604584
very low(<=0)      208158
mid(0.2<=0.4)      112894
high(0.4<=0.6)      20649
very high(0.6<)      3603
Name: sts_q, dtype: int64

In [69]:
df.loc[df['sts_q'] == 'high(0.4<=0.6)',['topic','stdn','sts']]

,topic,stdn,sts
14,방송/연예,아~ 근데 너는 뭐~ 요즘 보는 방송이나 빠진 연예인 같은 거 없는가?,0.400677
37,음악,난 약간 쎈 노래를 좋아해 막,0.467484
79,영화,요즘 그러니까 영화는 내가 왜 영화를 주제로 정했을까 언니?,0.567335
99,계절,우리는 가을 겨울 막 단감 이렇게 있고 막 그러잖아.,0.401814
118,계절,농사에는 이제 풍년이고 하는데 이제 계절 이런 것은 보면은,0.412722
...,...,...,...
949700,군대,뭐~ 군대 근무하실 때 조금 특별하게 기억에 나는 에피소드라던지,0.529238
949725,가족,그리고 또 우리 어머니는 나이 들어서 음식을,0.400559
949762,계절,너는 제일 싫어하는 계절이 뭐야?,0.424961
949838,아르바이트,지금 생각해 보니까 그렇게 좋았던 알바가 없는 거 같아.,0.404958


In [75]:
indx = ['reg','stdn', 'dial', 'eng','eojeol_stdn','sts', 'sts_q', 'title', 'year', 'date','category', 'topic', 'age', 'sex', 'prin_res', 'curr_res', 'age2', 'sex2',
       'prin_res2', 'curr_res2']

In [77]:
df[indx].to_csv(directory +'/meta/meta_raw_eng_sts_v1.csv', index=False)